# Introduction/Business Problem

In [4]:
# As more and more people are considering immigration to other countries
# this project would wish to give suggestions to whom would try to start their business in other countries
# in order to immigrate 
# So,
# lets see which place in New York would be recommended to start business

# Data 

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: \ 
  - anaconda/osx-64::ca-certificates-2018.03.07-0, anaconda/osx-64::openssl-1.0.2o-h26aff7b_0
  - anaconda/osx-64::ca-certificates-2018.03.07-0, defaults/osx-64::openssl-1.0.2o-h26aff7b_0
  - anaconda/osx-64::openssl-1.0.2o-h26aff7b_0, defaults/osx-64::ca-certificates-2018.03.07-0
  - defaults/osx-64::ca-certificates-2018.03.07-0, defaults/osx-64::openssl-1.0.2o-h26aff7bdone

# All requested packages already installed.

Solving environment: / 
  - anaconda/osx-64::ca-certificates-2018.03.07-0, anaconda/osx-64::openssl-1.0.2o-h26aff7b_0
  - anaconda/osx-64::ca-certificates-2018.03.07-0, defaults/osx-64::openssl-1.0.2o-h26aff7b_0
  - anaconda/osx-64::openssl-1.0.2o-h26aff7b_0, defaults/osx-64::ca-certificates-2018.03.07-0
  - defaults/osx-64::ca-certificates-2018.03.07-0, defaults/osx-64::openssl-1.0.2o-h26aff7bdone

# All requested packages already installed.

Folium installed
Libraries imported.


In [20]:
CLIENT_ID = '320KARHINECZ0UGN4DLW01ZC1PCQX0FLJWH222UHX4DQRACT' # your Foursquare ID
CLIENT_SECRET = 'IXGI2KILN3LYRO4HDIU2FCRYKTCBKLESRNPSXYUBDRFIASAW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 320KARHINECZ0UGN4DLW01ZC1PCQX0FLJWH222UHX4DQRACT
CLIENT_SECRET:IXGI2KILN3LYRO4HDIU2FCRYKTCBKLESRNPSXYUBDRFIASAW


In [21]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [22]:
latitude = 40.7127281
longitude = -74.0060152
radius = 500

In [23]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=320KARHINECZ0UGN4DLW01ZC1PCQX0FLJWH222UHX4DQRACT&client_secret=IXGI2KILN3LYRO4HDIU2FCRYKTCBKLESRNPSXYUBDRFIASAW&ll=40.7127281,-74.0060152&v=20180604&radius=500&limit=50'

In [24]:
import requests

In [25]:
results = requests.get(url).json()
'There are {} around NY.'.format(len(results['response']['groups'][0]['items']))

'There are 50 around NY.'

In [26]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57f0689d498e7d49d9189369',
  'name': 'The Bar Room at Temple Court',
  'location': {'address': '123 Nassau St',
   'lat': 40.7114477287544,
   'lng': -74.00680157032005,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.7114477287544,
     'lng': -74.00680157032005}],
   'distance': 157,
   'postalCode': '10038',
   'cc': 'US',
   'neighborhood': 'Financial District',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['123 Nassau St',
    'New York, NY 10038',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1d5941735',
    'name': 'Hotel Bar',
    'pluralName': 'Hotel Bars',
    'shortName': 'Hotel Bar',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_bar_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'gr

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/Users/qqho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,The Bar Room at Temple Court,Hotel Bar,123 Nassau St,40.711448,-74.006802,"[{'label': 'display', 'lat': 40.7114477287544,...",157,10038,US,Financial District,New York,NY,United States,"[123 Nassau St, New York, NY 10038, United Sta...",NaN,57f0689d498e7d49d9189369
1,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,40.711173,-74.006702,"[{'label': 'display', 'lat': 40.7111725, 'lng'...",182,10038,US,NaN,New York,NY,United States,"[123 Nassau St (Beekman St), New York, NY 1003...",Beekman St,56d8c0f8498edb854f926e6a
2,Alba Dry Cleaner & Tailor,Laundry Service,140 Nassau St,40.711434,-74.006272,"[{'label': 'display', 'lat': 40.71143355866638...",145,10038,US,NaN,New York,NY,United States,"[140 Nassau St (Beekman and Spruce st), New Yo...",Beekman and Spruce st,4c606c3e1e5cd13ad1a1a1ed
3,City Hall Park,Park,17 Park Row,40.712415,-74.006724,NaN,69,10038,US,NaN,New York,NY,United States,"[17 Park Row (btwn Broadway & Centre St), New ...",btwn Broadway & Centre St,3fd66200f964a520d8f11ee3
4,Augustine,French Restaurant,5 Beekman St,40.711310,-74.006660,"[{'label': 'display', 'lat': 40.71130993467075...",166,10038,US,NaN,New York,NY,United States,"[5 Beekman St, New York, NY 10038, United States]",NaN,58191674ded8f8626ed70af0
5,The Wooly Daily,Coffee Shop,11 Barclay St,40.712137,-74.008395,"[{'label': 'display', 'lat': 40.71213724343698...",211,10007,US,NaN,New York,NY,United States,"[11 Barclay St (Broadway), New York, NY 10007,...",Broadway,56093809498e5344ab8835a6
6,The Class by Taryn Toomey,Gym / Fitness Center,22 Park Pl,40.712753,-74.008734,"[{'label': 'display', 'lat': 40.71275251771485...",229,10007,US,NaN,New York,NY,United States,"[22 Park Pl, New York, NY 10007, United States]",NaN,58a253c830ecc66c9e5b40a0
7,Gibney Dance Center Downtown,Dance Studio,280 Broadway,40.713923,-74.005661,"[{'label': 'display', 'lat': 40.71392316324669...",136,10007,US,NaN,New York,NY,United States,"[280 Broadway (Chambers St), New York, NY 1000...",Chambers St,53373f26498e940581c90985
8,CrossFit 212 TriBeCa,Gym,281 Broadway,40.714537,-74.005999,"[{'label': 'display', 'lat': 40.71453743117048...",201,10007,US,NaN,New York,NY,United States,"[281 Broadway, New York, NY 10007, United States]",NaN,52001eed498e9ac16ca5e20b
9,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",256,10007,US,NaN,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",at Church St,4c154c9a77cea593c401d260
